In [2]:
from matplotlib import pyplot as plt
import numpy as np
import math
import meep as mp
import cmath

shape_size = 48

sx, sy, sz = 1, 1, 4
h = 1.25
dpml = 0.5
b_m, c_m = 1.4, 3.54
res = 15
echo = 1000
cell_size = mp.Vector3(sx,sy,sz)
fcen = 0.5
df = 0.2
theta = math.radians(0)
nfreq = 200

# k with correct length (plane of incidence: XZ) 
k = mp.Vector3(math.sin(theta),0,math.cos(theta)).scale(fcen)
def pw_amp(k, x0):
    def _pw_amp(x):
        return cmath.exp(1j * 2 * math.pi * k.dot(x + x0))
    return _pw_amp

def get_trans(vertices):
    geometry = [mp.Block(size = cell_size, material=mp.Medium(index=b_m)),
                mp.Prism(vertices, 
                         height=h, 
                         material=mp.Medium(index=c_m),
                         center=mp.Vector3()
                        )]
    pml_layers = [mp.PML(thickness=1, direction = mp.Z, side=mp.High),
                  mp.Absorber(thickness=1,direction = mp.Z, side=mp.Low)]
    src_pos = -(sz/2 - dpml - 0.5)
    src = [mp.Source(src = mp.GaussianSource(fcen, fwidth=df),
                     component = mp.Ey,
                     center = mp.Vector3(0,0,src_pos),
                     size = mp.Vector3(sx,sy,0),
                     amp_func=pw_amp(k,mp.Vector3(0,0,src_pos)))]
    sim = mp.Simulation(resolution=res,
                        cell_size=cell_size,
                        boundary_layers=pml_layers,
                        sources=src,
                        geometry=geometry,
                        k_point=k)
    freg = mp.FluxRegion(center=mp.Vector3(0,0,-src_pos),
                         size = mp.Vector3(sx,sy,0))
    trans = sim.add_flux(fcen, df, nfreq, freg)
    sim.run(until = echo)
    bend = mp.get_fluxes(trans)
    
    #get straight
    sim.reset_meep()
    geometry = [mp.Block(size = cell_size, material=mp.Medium(index=b_m))]
    pml_layers = [mp.PML(thickness= 1, direction = mp.Z, side=mp.High),
                  mp.Absorber(thickness=1,direction = mp.Z, side=mp.Low)]
    src = [mp.Source(src = mp.GaussianSource(fcen, fwidth=df),
                     component = mp.Ey,
                     center = mp.Vector3(0,0,src_pos),
                     size = mp.Vector3(sx,sy,0),
                     amp_func=pw_amp(k,mp.Vector3(0,0,src_pos)))]
    sim = mp.Simulation(resolution=res,
                        cell_size=cell_size,
                        boundary_layers=pml_layers,
                        sources=src,
                        geometry=geometry,
                        k_point=k)
    freg = mp.FluxRegion(center=mp.Vector3(0,0,-src_pos),
                         size = mp.Vector3(sx,sy,0))
    trans = sim.add_flux(fcen, df, nfreq, freg)
    sim.run(until = echo)
    straight = mp.get_fluxes(trans)
    flux_freqs = mp.get_flux_freqs(trans)
    
    c = 300
    p = 0.6
    Ts = []
    for i in range(nfreq):
        Ts = np.append(Ts, bend[i]/straight[i])
    return np.multiply(flux_freqs, c/p),Ts

In [3]:
def data_generator(batch):
    data = []
    # get shape from 'data/DATAX_sh.txt'
    coordinates = np.genfromtxt('data/DATA'+str(batch)+'_sh.txt')
    xc, yc = coordinates[:, 0], coordinates[:, 1]
    size = len(xc) // shape_size
    xc = np.reshape(xc, (size, shape_size))
    yc = np.reshape(yc, (size, shape_size))

    for i in range(size):
        # form shape
        vertices = [mp.Vector3(xc[i][0],yc[i][0])]
        for j in range(1, len(xc[i]) - 1):
            # eliminate duplicate point
            if xc[i][j] == xc[i][j - 1] and yc[i][j] == yc[i][j - 1]:
                continue
            vertices.append(mp.Vector3(xc[i][j], yc[i][j]))
            print(xc[i][j], yc[i][j])
        # calculate transmission
        _, Ts = get_trans(vertices)
        data.append(Ts)
    # save the spectrum to 'data/DATAX_sp.txt'
    np.savetxt('data/DATA'+str(batch)+'_sp.txt', data)
    x = np.genfromtxt('data/SP_xaxis.txt')
    y = np.genfromtxt('data/DATA'+str(batch)+'_sp.txt')
    for i in range(size):
        print('The '+str(i)+'th shape:')
        plt.axis('equal')
        plt.ylim(-0.5, 0.5)
        plt.xlim(-0.5, 0.5)
        plt.plot(xc[i], yc[i])
        plt.fill(xc[i], yc[i])
        plt.show()
        plt.ylim(0, 1.1)
        plt.plot(x, y[i])
        plt.show()

In [4]:
start, end = 5, 6
for i in range(start, end):
    data_generator(i)

0.089100652418837 0.014112156965909
0.089100652418837 0.028950556918082
0.089100652418837 0.045399049973955
0.077901181240045 0.056598521152746
0.067249851196396 0.067249851196396
0.056598521152746 0.077901181240045
0.045399049973955 0.089100652418837
0.028950556918082 0.089100652418837
0.014112156965909 0.089100652418837
0.0 0.089100652418837
-0.014112156965909 0.089100652418837
-0.028950556918082 0.089100652418837
-0.045399049973955 0.089100652418837
-0.056598521152746 0.077901181240045
-0.067249851196396 0.067249851196396
-0.077901181240045 0.056598521152746
-0.089100652418837 0.045399049973955
-0.089100652418837 0.028950556918082
-0.089100652418837 0.014112156965909
-0.089100652418837 0.0
-0.089100652418837 -0.014112156965909
-0.089100652418837 -0.028950556918082
-0.089100652418837 -0.045399049973955
-0.077901181240045 -0.056598521152746
-0.067249851196396 -0.067249851196396
-0.056598521152746 -0.077901181240045
-0.045399049973955 -0.089100652418837
-0.028950556918082 -0.0891006524

KeyboardInterrupt: 

In [ ]:
LINE = 6
from math import sin, cos, pi
d_to_r = pi/180
LINE = 6 
ANGLE = [0, 9,18,27,36, 45]
SYMMETRY = [(-1, 1), (-1, -1), (1, -1)]
def polar_to_xy(polar):
    coor = []
    for i in range(LINE):
        x = cos(ANGLE[i] * d_to_r) * polar[i]
        y = sin(ANGLE[i] * d_to_r) * polar[i]
        coor.append([x, y])
    for i in range(LINE-1, -1, -1):
        coor.append([coor[i][1], coor[i][0]])
    quarter = 1
    for dx, dy in SYMMETRY:
        if quarter%2 == 1:
            for i in range(LINE*2 - 1, -1, -1):
                coor.append([coor[i][0]*dx, coor[i][1]*dy])
        else:
            for i in range(LINE*2):
                coor.append([coor[i][0]*dx, coor[i][1]*dy])
        quarter += 1 
    return coor

P_shape = [0.14291796, 0.13878718, 0.14108998, 0.14678332, 0.14327656, 0.14059259]
T_shape = [0.09876883, 0.1       , 0.13169178, 0.2       , 0.19258231, 0.1902113]
shape = polar_to_xy(T_shape)
vertices = [mp.Vector3(shape[0][0], shape[0][1])] 
for i in range(1, len(shape)):
    # eliminate duplicate point
    #print(shape[i])
    if abs(shape[i][0] - shape[i-1][0]) < 1e-5 and abs(shape[i][1] - shape[i-1][1]) < 1e-5:
        continue
    vertices.append(mp.Vector3(shape[i][0], shape[i][1]))
get_trans(vertices)